# Práctica de lidar elástico

Éste es el código que se utilizará para el desarrollo de la práctica de lidar elástico. Para ello el/la alumno/a solo tendrá que modificar las sentencias que se encuentran entre:

#------------------------------------------------------------------------------------------------------------------------------

         Modifique SOLO aquí dentro
#------------------------------------------------------------------------------------------------------------------------------

Importante:
- Un error recurrente puede ser que las rutas se especifiquen con '\\' como separador entre carpetas, mientras que el lenguaje Python utiliza '\\\\' (doble). 
- También se recomienda correr una a una las celdas, y no correr de nuevo una celda mientras se está ejecutando (el software puede quedarse colgado). Si mientras una celda se está ejecutando, se realiza algún cambio en el código de esa celda y queremos volver a ejecutarla, deberemos esperar a que termine o reiniciar los kernels.
- Si lo desea, puede jugar con el resto del código.
- Para el desarrollo de la práctica y elaboración del informe se pueden emplear resultados obtenidos en otras prácticas, que sean relativos a la misma situación atmosférica.

**IMPORTACIÓN DE LIBRERÍAS Y ESPECIFICACIÓN DE RUTAS**

En primer lugar **importamos las librerias** necesarias. En el caso de que no se encuentre alguno de los módulos al correr esta celda tendremos que instalar dicho módulo en Anaconda Navigator -> Environments o a traves de Anaconda Prompt.

In [ ]:
import os
from pathlib import Path
from pdb import set_trace

from datetime import datetime
import numpy as np #For general mathematical calculations
import matplotlib.pyplot as plt #For plotting
from scipy.integrate import trapz
from scipy.signal import savgol_filter #Allows to smooth a profile through the Savitzky-Golay filter

#For working with lidar products (developed by the Atmospheric Physics Group of the UGR, GFAT)
from gfatpy.atmo import ecmwf, atmo

from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import date_quicklook
from gfatpy.lidar.quality_assurance.rayleigh_fit import rayleigh_fit_from_date
from gfatpy.lidar import utils
from gfatpy.lidar.retrieval import klett
from gfatpy.lidar.retrieval.helper import RetrievalHelper

from gfatpy.aeronet.reader import reader as aeronet_reader
from gfatpy.aeronet.utils import find_aod

from gfatpy.utils.plot import font_axes, color_list


%load_ext autoreload
%autoreload 2

%matplotlib inline

Tambien, antes de comenzar con el tratamiento de los ficheros, definimos la ruta en la que se encuentran los datos.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
databasepath = Path(r'G:\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_elastico\data\1a')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
databasepath = Path(r'C:\Users\Usuario\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_elastico\data\1a')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
outputpath =   Path(r'C:\Users\Usuario\Downloads\geomet_output')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

databasepath.is_dir(), outputpath.is_dir() #Comprueba que ambas carpetas existen.

**REPRESENTACIÓN DE SEÑAL CORREGIDA DE RANGO Y RETRODISPERSIÓN MOLECULAR**

En cualquier análisis de medidas lidar, el primer paso es **representar los perfiles de señal corregida de rango y retrodispersión molecular**, para hacernos una idea de la situación atmosférica, y también para poder razonar cual es el valor de altura de referencia que se tendrán que emplear en los cálculos posteriores.

En la celda de código siguiente representaremos el perfil de la señal corregida de rango, con tan solo los parámetros de entrada:
- file_path: será la ruta (incluyendo el nombre y extensión) del fichero que se desea leer. Será necesario tener en cuenta que los ficheros que vamos a trabajar tienen el formato "YYYYMMDDgrNN_CCCCCCC.nc" donde YYYYMMDD hace referencia al año, mes y día respectivamente; NN hace referencia al número de la medida para la fecha dada, es decir, para la primera medida del día (de 30 min) NN toma el valor 00, para la segunda medida NN toma el valor 01 y así sucesivamente; y finalmente CCCCCCC se corresponde con el identificador al canal, que toma el valor 0000760 para el canal de 532 nm y 0001199 para 355 nm.
- ymin, y_max: las alturas mínima y máxima, respectivamente, dadas en metros asl para representar el perfil de señal corregida de rango. Son parámetros opcionales, pero pueden ser útiles.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
qlpath = outputpath / 'quicklooks' #Output quicklook folder path. It is suggested to mantain the proposed path
ini_date = '20210719' #Data for which quicklooks will be obtained. Format yyyymmdd
end_date = ini_date
#------------------------------------------------------------------------------------------------------------------------------
channels = ["532xpa"]
date_quicklook(
    ini_date,
    channels2plot=channels,
    path1a=databasepath,
    figpath=qlpath,
    colorbar_range=(0,5e6),
)

A continuación representaremos el perfil de la RCS normalizado junto con el de retrodispersión molecular, con tan solo los parámetros de entrada:
- file_path.
- y_min, y_max.
- y_norm_min, y_norm_max: las alturas en metros asl entre las cuales normalizar el perfil de la RCS, de manera que y_norm_min se tendrá que corresponder (muy estimativamente) con una altura para la que nos encontremos en atmósfera molecular. Lo ideal es utilizar una ventana de un kilómetro sobre y_norm_min para realizar esta normalización del perfil de la RCS, aunque puede ser interesante probar diferentes situaciones. Para una mejor visualización de la normalización de la RCS, se recomienda que y_max sea mayor que y_norm_max.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
min_reference_height = 6500     #Sets the value of the minimum reference altitude in meters asl
max_reference_height = 7000    #Sets the value of the maximum reference altitude in meters asl
smooth_window = 250
max_profile_range = 30000
min_profile_range= 0
#------------------------------------------------------------------------------------------------------------------------------

channels = ['355xta', "532xta", '1064xta']

rf_ds = rayleigh_fit_from_date(
    datetime(2021, 7, 19),
    initial_hour=3,
    lidar_name="MULHACEN",
    channels=channels,
    min_reference_height = min_reference_height,
    max_reference_height =max_reference_height,
    smooth_window = smooth_window,
    max_profile_range = max_profile_range,
    min_profile_range= min_profile_range,
    meteorology_source= "ecmwf",
    dir_1a=databasepath,    
    output_dir=outputpath,
    save_fig=False,
)

**MÉTODO DE KLETT-FERNALD**

El desarrollo principal de la práctica se basará en las siguientes celda, donde **especificaremos diferentes valores de altura de referencia y razón lidar para realizar la inversión de Klett**, que son los inputs requeridos en este método. Es necesario tener en cuenta las peculiaridades de cada uno de estos parámetros necesarios.
- file_path.
- in_h_ref: será la altura de refencia en metros asl utilizada en el proceso de inversión. Por las necesidades del usuario, se permite que in_h_ref sea tanto un único valor (por ejemplo in_h_ref = 7000) como una lista de valores (por ejemplo in_h_ref = [4000, 5000, 6000]). 
- in_lr: será el valor de la razón lidar utilizado en el proceso de inversión. Por las necesidades del usuario, se permite que in_lr sea tanto un único valor (por ejemplo in_lr = 50) como una lista de valores (por ejemplo in_lr = [40, 50, 60]).
- **IMPORTANTE: el programa no permite utilizar simultáneamente una lista de entrada en in_h_ref e in_lr**.
- klett_window: será el valor de la ventana en metros utilizada como referencia en la inversión de Klett. Se utilziará la altura de referencia para centrar dicha ventana. Ten en cuenta que ventanas muy estrechas pueden dar fallos en el proceso. 
- y_min, y_max.

Se recomienda que en primer lugar se experimente el comportamiento de la inversión de Klett-Fernald especificando diferentes parejas de valores, pero en ejecuciones distintas de la celda, de in_h_ref e in_lf; es decir, no utilizar ninguna lista de valores. Posteriormente, se deberían especificar varios valores de una variable junto con un único valor de la otra para poder comparar los diferentes perfiles.

In [ ]:
# Lectura de datos
#------------------------------------------------------------------------------------------------------------------------------
filepath = databasepath / '2019'/ '07'/ '11' / 'mhc_1a_Prs_rs_xf_20190711.nc'   #Lidar product path. Example : 'C:\\Users\\user\\Desktop\\data\\1a\\2020\\01\\01\\mhc_1a_Prs_rs_xf_20200101.nc' 
dcfilepath = databasepath / '2019'/ '07'/ '11' / 'mhc_1a_Pdc_rs_xf_20190711_0558.nc' #Lidar dark current product path
ini_date = '20190711T100000'   #Initial date and time. Format 'yyyymmddThhmmss', where T represents literally the letter T. Example: '20200101T200000'
end_date = '20190711T103000'   #Same format than ini_date
#------------------------------------------------------------------------------------------------------------------------------

if os.path.exists(databasepath) and os.path.exists(dcfilepath):
    channels = ["532xta"]
    data = preprocess(filepath, dc_fl=dcfilepath, channels=channels, ini_date=ini_date, end_date=end_date)
else:
    print("One (or both) of the given files can not be found.")

In [ ]:
# Molecular elastic
z = data.range.values
ecmwf_data = ecmwf.get_ecmwf_temperature_pressure(
    datetime(2018, 5, 11), ranges_array=z
)
atmo_data = {}
for wavelength in [355, 532, 1064]:
    atmo_data[wavelength] = atmo.molecular_properties(
        wavelength, ecmwf_data.pressure, ecmwf_data.temperature, heights=z
    )

beta_mol532 = atmo_data[532]["molecular_beta"].values
alpha_mol532 = atmo_data[532]["molecular_alpha"].values

In [ ]:
#Inversión de 532nm
#------------------------------------------------------------------------------------------------------------------------------
ymin = 6000     #Sets the value of the minimum reference altitude in meters asl
ymax = 7000    #Sets the value of the maximum reference altitude in meters asl
aer_lr = 45    #Sets the value(s) of the lidar ratio

#------------------------------------------------------------------------------------------------------------------------------
#Promedio temporal de la señal
signal_532xta = data['signal_532xta'].mean('time')

#Conversión a range-corrected signal
rcs_532xta = utils.signal_to_rcs(signal_532xta, z).values

#Algorithm Klett
beta532 = klett.klett_rcs(rcs_532xta, 
                range_profile = z, 
                beta_mol_profile=beta_mol532,
                lr_aer= aer_lr,                
                ymin = ymin,
                ymax = ymax)

#Guardado de la señal en el dataset creado anteriormente
data['beta_532xta'] = (('range'), beta532*1e6)
data['beta_532xta'] = data['beta_532xta'].assign_attrs({'long_name': '$\\beta_{part}$', 
                                  'units':'$Mm^{-1}·sr^{-1}$'})
#Representación de la retrodispersión a 532 nm
fig, ax = plt.subplots(figsize=[3,5])
data['beta_532xta'].plot(y='range',ax=ax, c= 'g')
ax.set_ylim(0,7500)
ax.set_xlim(0,4)

In [ ]:
#Inversión de 355, 532 y 1064nm
#------------------------------------------------------------------------------------------------------------------------------
ymin = 6000     #Sets the value of the minimum reference altitude in meters asl
ymax = 7000    #Sets the value of the maximum reference altitude in meters asl
aer_lr = 45    #Sets the value(s) of the lidar ratio
#------------------------------------------------------------------------------------------------------------------------------
channels = ['355xta', "532xta", '1064xta']
data = preprocess(filepath, dc_fl=dcfilepath, 
                  channels=channels, ini_date=ini_date, end_date=end_date)

for wavelength in  [355, 532, 1064]:
    signal_ = data[f'signal_{wavelength}xta'].mean('time')
    rcs_ = utils.signal_to_rcs(signal_, z).values    
    beta_mol_ = atmo_data[wavelength]["molecular_beta"].values
    beta_ = klett.klett_rcs(rcs_, 
                    range_profile = z, 
                    beta_mol_profile=beta_mol_,
                    lr_aer= aer_lr,                
                    ymin = ymin,
                    ymax = ymax)
    
    data[f'beta_{wavelength}xta'] = (('range'), beta_)
    
    #Guardado de la señal en el dataset creado anteriormente
    data[f'beta_{wavelength}xta'] = (('range'), beta_*1e6)
    data[f'beta_{wavelength}xta'] = data[f'beta_{wavelength}xta'].assign_attrs({
                                                'long_name': '$\\beta_{part}$', 
                                                 'units':'$Mm^{-1}·sr^{-1}$', 
                                                'lr_aer_sr': aer_lr, 
                                                 'reference_range_mr': f'{ymin}-{ymax}'})


In [ ]:
#Representación de la retrodispersión 
colors = {355: 'b', 532: 'g', 1064: 'r'}
fig, ax = plt.subplots(figsize=[3,5])
for wavelength in  [355, 532, 1064]:
    data[f'beta_{wavelength}xta'].plot(y='range',ax=ax, label=f'{wavelength} nm', c=colors[wavelength])
ax.set_ylim(0,7500)
ax.set_xlim(0,4)
ax.legend()

In [ ]:
#Inversión de 35, 532 para varias razonres lidar
#------------------------------------------------------------------------------------------------------------------------------
ymin = 6000     #Sets the value of the minimum reference altitude in meters asl
ymax = 7000    #Sets the value of the maximum reference altitude in meters asl
aer_lr = np.arange(20,80,5)    #Sets the value(s) of the lidar ratio
#------------------------------------------------------------------------------------------------------------------------------
for wavelength in  [355, 532]:
    for lr_ in aer_lr:
        signal_ = data[f'signal_{wavelength}xta'].mean('time')
        rcs_ = utils.signal_to_rcs(signal_, z).values    
        beta_mol_ = atmo_data[wavelength]["molecular_beta"].values
        beta_ = klett.klett_rcs(rcs_, 
                        range_profile = z, 
                        beta_mol_profile=beta_mol_,
                        lr_aer= lr_,                
                        ymin = ymin,
                        ymax = ymax)

        data[f'beta_{wavelength}xta_{lr_}'] = (('range'), beta_)

        #Guardado de la señal en el dataset creado anteriormente
        data[f'beta_{wavelength}xta_{lr_}'] = (('range'), beta_*1e6)
        data[f'beta_{wavelength}xta_{lr_}'] = data[f'beta_{wavelength}xta_{lr_}'].assign_attrs({
                                                    'long_name': '$\\beta_{part}$', 
                                                     'units':'$Mm^{-1}·sr^{-1}$', 
                                                    'lr_aer_sr': aer_lr, 
                                                     'reference_range_mr': f'{ymin}-{ymax}'})

#Representación de la retrodispersión 
colors = color_list(len(aer_lr))
fig, ax = plt.subplots(figsize=[3,5])
wavelength = 532
for idx, lr_ in enumerate(aer_lr):     
    data[f'beta_532xta_{lr_}'].plot(y='range',ax=ax, label=f'{lr_} sr', c=colors[idx])
ax.set_ylim(0,7500)
ax.set_xlim(0,4)
ax.legend(fontsize=6)
ax.set_title('¿Cuál es la razón lidar correcta?')        

In [ ]:
range_resolution = z[1]-z[0]
aod_with_lidar = {355: [], 532: []}
for wavelength in [355, 532]:    
    for idx, lr_ in enumerate(aer_lr):     
        alfa = data[f'beta_{wavelength}xta_{lr_}'].values*1e-6*lr_        
        alfa[alfa<0] = 0
        try:
            aod_with_lidar[wavelength].append(trapz(alfa, dx=range_resolution)) 
        except:
            set_trace()

In [ ]:
colors = {355: 'b', 532: 'g', 1064: 'r'}
fig, ax = plt.subplots(figsize=[10,3])
for wavelength in [355, 532]:    
    plt.scatter(aer_lr ,aod_with_lidar[wavelength],c=colors[wavelength], label=f'{wavelength} nm')
ax.set_xlabel('AOD, [#]')
ax.set_ylabel('LR, [sr]')
ax.set_title('¿Cuál es la razón lidar correcta?')
ax.legend()

**USO DE FOTOMETRÍA SOLAR PARA LA ELECCIÓN DE LA RAZÓN LIDAR**

En la siguiente celta **utilizaremos los datos de fotometría solar para una correcta elección del valor de la razón lidar**. Para ello, en primer lugar, lo que haremos será leer las medidas de AOD (disponibles en AERONET, con extensión '.lev20') del fotómetro solar contiguo al lidar. A partir de la expresión mostrada en la teoría podremos calcular el AOD obtenido con el lidar para diferentes combinaciones de razón lidar y altura de referencia y podremos seleccionar un valor apropiado comparando con el AOD obtenido por el fotómetro solar.

En la celda de código siguiente se leerá el fichero descargado de AERONET y se representrán las medidas disponibles en dicho fichero. Para ello, solamente se tendrá que expecificar:
- AERONET_file_path: será la ruta (incluyendo el nombre y extensión) del fichero de AERONET que se desea leer. Preferentemente se deben emplear datos de nivel 2.0 (ficheros con extensión '.lev20'), aunque los de nivel 1.5 (extensión '.lev15') también son válidos.
- ymin, y_max: los valores mínimo y máximo, respectivamente, del eje vertical, es decir, los valores de AOD. Para una mejor interpretación de los valores, y_min suele fijarse a 0.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
aeronet_filepath = r"C:\Users\Usuario\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_elastico\data\20040101_20211231_Granada.all"
target_datetime = datetime(2019, 7, 11, 10, 15, 0)
wavelenght = 355
allowed_time_gap_hour = 4
#-------------------------------------------------------------------------------------------------------------------------------
header, aeronet_dataframe = aeronet_reader(aeronet_filepath)

found_sun_photometer_aod, measure_datetime = find_aod(
    wavelenght, 
    aeronet_dataframe, 
    header, 
    target_datetime=target_datetime, 
    allowed_time_gap_hour=allowed_time_gap_hour,
    allowed_interpolation=True
)

if found_sun_photometer_aod is not None: 
    print(f'Found AOD at {wavelenght} nm:{np.round(found_sun_photometer_aod,3)}')
    print(f'Measurement datetime: {measure_datetime}')

    # Check distance between both    
    if target_datetime > nearest_datetime:
        time_gap = target_datetime - nearest_datetime
    else:
        time_gap = nearest_datetime - target_datetime
    time_gap_hour = time_gap.seconds / 60.0

    found_sun_photometer_aod_355 = found_sun_photometer_aod
    print(f'Time gap: {np.round(time_gap_hour,3)} minutes')
else:
    print('Not found')

 

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
aeronet_filepath = r"C:\Users\Usuario\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_elastico\data\20040101_20211231_Granada.all"
target_datetime = datetime(2019, 7, 11, 10, 15, 0)
wavelenght = 532
allowed_time_gap_hour = 4
#-------------------------------------------------------------------------------------------------------------------------------
header, aeronet_dataframe = aeronet_reader(aeronet_filepath)

found_sun_photometer_aod, measure_datetime = find_aod(
    wavelenght, 
    aeronet_dataframe, 
    header, 
    target_datetime=target_datetime, 
    allowed_time_gap_hour=allowed_time_gap_hour,
    allowed_interpolation=True
)

if found_sun_photometer_aod is not None: 
    print(f'Found AOD at {wavelenght} nm:{np.round(found_sun_photometer_aod,3)}')
    print(f'Measurement datetime: {measure_datetime}')

    # Check distance between both
    if (measure_datetime - target_datetime).seconds > 0:
        time_gap = measure_datetime - target_datetime
    else:
        time_gap = target_datetime - measure_datetime
    time_gap_hour = time_gap.seconds / 60.0 
    
    found_sun_photometer_aod_532 = found_sun_photometer_aod
    print(f'Time gap: {np.round(time_gap_hour,3)} minutes')
else:
    print('Not found')

 

A continuación, estimaremos el valor razón lidar a partir de los valores de AOD de AERONET. Para ello, se correrá la inversión de Klett-Fernald para una lista de posibles valores de altura de referencia y razón lidar, de manera que obtendremos un AOD medido por el lidar, que compararemos con el AOD de AERONET para **estimar** el valor de razón lidar más apropiado. Será necesario tener en cuenta los siguientes parámetros de entrada:
- AERONET_file_path, lidar_file_path.
- in_h_ref, in_lr: serán las listas de altura de referencia en metros asl y razón lidar, y para cada pareja de valores se obtiene el perfil de retrodispersión de partículas, se multiplica por la razón lidar para obtener el perfil de extinción de partículas y finalmente se integra el perfil, desde superficie hasta la altura de referencia, para calcular el AOD. En la región de solapamiento incompleto se asume un valor constante de coeficiente de extinción de partículas correspondiente al primer valor fiable (en la altura de solapamiento completo).
- **IMPORTANTE: En este caso sí se permite simultáneamente listas de in_h_ref e in_lr.**
- z_overlap: será la altura en metros asl a la cual se tiene solapamiento completo de la señal lidar.
- z_station: será la altitud en metros asl a la que se encuentra la estación.

A la salida tendremos 3 tablas distintas: 
- tabla 1: valor de AOD obtenido por el lidar en la longitud de onda dada.
- tabla 2: diferencia entre el AOD obtenido por el lidar y el AOD de referencia del fotómetro solar. 
- tabla 3: porcentaje del AOD del lidar con el que contribuye la región de solapamiento incompleto. 

Find AOD_{'sun photometer'} in AOD array from lidar.

In [ ]:
np.max(aod_with_lidar[355] + aod_with_lidar[532])

In [ ]:
idx_min_355 = np.abs(aod_with_lidar[355] - found_sun_photometer_aod_355).argmin()
print(f'AOD at 355 nm with lidar: {np.round(aod_with_lidar[355][idx_min_355],2)}')
print(f'AOD at 355 nm with sun photometer: {np.round(found_sun_photometer_aod_355,2)}')
print(f'Lidar ratio [sr] with best AOD match: {aer_lr[idx_min_355]}')

idx_min_532 = np.abs(aod_with_lidar[532] - found_sun_photometer_aod_532).argmin()
print(f'AOD at 532 nm with lidar: {np.round(aod_with_lidar[532][idx_min],2)}')
print(f'AOD at 532 nm with sun photometer: {np.round(found_sun_photometer_aod_532,2)}')
print(f'Lidar ratio [sr] with best AOD match: {aer_lr[idx_min_532]}')

fig, ax = plt.subplots(figsize=[10,3])
for wavelength in [355, 532]:    
    plt.scatter(aer_lr,aod_with_lidar[wavelength],c=colors[wavelength], label=f'{wavelength} nm')
plt.hlines(found_sun_photometer_aod_355,aer_lr[0],aer_lr[-1],ls='dashed',color='b', label='AOD_{sf}[355 nm]')
plt.hlines(found_sun_photometer_aod_532,aer_lr[0],aer_lr[-1],ls='dashed',color='g', label='AOD_{sf}[355 nm]')
plt.vlines(aer_lr[idx_min_355],0,aod_with_lidar[355][idx_min_355],ls='solid',color='b',label='Best match')
plt.vlines(aer_lr[idx_min_532],0,aod_with_lidar[532][idx_min_532],ls='solid',color='g',label='Best match')
ax.set_ylim(0,np.max(aod_with_lidar[355] + aod_with_lidar[532]))
ax.set_ylabel('AOD, [#]')
ax.set_xlabel('LR, [sr]')
ax.set_title('¿Cuál es la razón lidar correcta?')
ax.legend(loc='lower right')

**PERFILES DE EXPONENTE DE ANGSTRÖM**

Finalmente **estimaremos los perfiles de exponente de Ångström utilizando los canales de 355 nm y 532 nm**. Debemos tener en cuenta que se realizarán dos inversiones de Klett distintas, una para la señal en 355 nm y otra para la señal en 532 nm, por lo que tendremos que especificar los siguientes parámetros de entrada:
- file_path_532, in_h_ref_532, in_lr_532, klett_window_532: de acuerdo a las explicaciones de las celdas anteriores, estos serán los parámetros relativos al canal de 532 nm.
- file_path_355, in_h_ref_355, in_lr_355, klett_window_355: de acuerdo a las explicaciones de las celdas anteriores, estos serán los parámetros relativos al canal de 355 nm.
- y_min, y_max.

In [ ]:
beta_355 = data['beta_355xta_55'].where(data['beta_355xta_55']>0).values
beta_532 = data['beta_532xta_40'].where(data['beta_532xta_40']>0).values
angs = -np.log(beta_355/beta_532)/np.log(355/532)
#Guardado de la propiedad en el dataset creado anteriormente
data['angstrom_355_532'] = (('range'), angs)
data['angstrom_355_532'] = data['angstrom_355_532'].assign_attrs({
                                            'long_name': '$\\AE_{part}[355-532 nm]$', 
                                             'units':'#'})
fig, ax = plt.subplots(1,2,figsize=[10,7])
ax1 = plt.subplot(1,2,1)
#Representación de la retrodispersión 
colors = {355: 'b', 532: 'g'}
for wavelength in  [355, 532]:
    data[f'beta_{wavelength}xta_40'].plot(y='range',ax=ax1, label=f'{wavelength} nm', c=colors[wavelength])
ax1.set_ylim(0,7500)
ax1.set_xlim(0,4)
ax1.legend()
ax2 = plt.subplot(1,2,2)
data[f'angstrom_355_532'].plot(y='range',ax=ax2, c='r')
ax2.set_ylim(0,7500)
ax2.set_xlim(-2,4)
# ax.legend(fontsize=6)
# ax.set_title('¿Cuál es la razón lidar correcta?')  